In [28]:
def assign_attack_type(label):
        attack_dict = { 'normal': 'normal',
                                   
                'neptune': 'dos', 'back': 'dos', 'land': 'dos',
                'pod': 'dos', 'smurf': 'dos', 'teardrop': 'dos', 'mailbomb': 'dos',
                'apache2': 'dos', 'processtable': 'dos', 'udpstorm': 'dos', 'worm': 'dos',

                'ipsweep': 'probe', 'nmap': 'probe', 'portsweep': 'probe', 'satan': 'probe',
                'mscan': 'probe', 'saint': 'probe', 

                'ftp_write': 'r2l', 'guess_passwd': 'r2l','imap': 'r2l', 'multihop': 'r2l', 'phf': 'r2l', 'spy': 'r2l', 'warezclient': 'r2l',
                'warezmaster': 'r2l', 'sendmail': 'r2l', 'named': 'r2l', 'snmpgetattack': 'r2l',
                'snmpguess': 'r2l', 'xlock': 'r2l', 'xsnoop': 'r2l', 'httptunnel': 'r2l',

                'buffer_overflow': 'u2r', 'loadmodule': 'u2r', 'perl': 'u2r', 'rootkit': 'u2r',
                'ps': 'u2r', 'sqlattack': 'u2r', 'xterm': 'u2r'
                }
        return attack_dict[label]

class Dataset:    
    def __init__(self, data):
        self.data = data

    def get_data(self):
        return self.data
    
    def get_label2(self):
        self.label2 = self.data.copy()
        self.label2['label'] = self.label2['label'].apply(lambda x: 'normal' if x == 'normal' else 'attack')
        return self.label2
    
    def get_label5(self):
        self.label5 = self.data.copy()
        self.label5['label'] = self.label5['label'].apply(lambda x: assign_attack_type(x))
        return self.label5


In [29]:
import pandas as pd

train_df = pd.read_csv(f'KDDTrain+.txt', header=None)
test_df = pd.read_csv(f'KDDTest+.txt', header=None)

columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
           'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted',
           'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
           'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
           'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
           'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
           'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
           'dst_host_srv_rerror_rate', 'label', 'score']
train_df.columns = columns
test_df.columns = columns

nominal_features = ['protocol_type', 'service', 'flag']
binary_features = ['land', 'logged_in', 'root_shell', 'su_attempted', 'is_host_login', 'is_guest_login']
numeric_features = [feature for feature in columns if feature not in nominal_features + binary_features + ['label', 'score']]


In [30]:
train_data = Dataset(train_df)
test_data = Dataset(test_df)

print(train_data.get_label2()['label'].value_counts())
print(test_data.get_label2()['label'].value_counts())  

print(train_data.get_label5()['label'].value_counts())
print(test_data.get_label5()['label'].value_counts())


normal    67343
attack    58630
Name: label, dtype: int64
attack    12833
normal     9711
Name: label, dtype: int64
normal    67343
dos       45927
probe     11656
r2l         995
u2r          52
Name: label, dtype: int64
normal    9711
dos       7460
r2l       2885
probe     2421
u2r         67
Name: label, dtype: int64


In [34]:
print(train_data.get_label2().groupby(['protocol_type', 'label']).size())
print(test_data.get_label2().groupby(['protocol_type', 'label']).size())

print(train_data.get_label5().groupby(['protocol_type', 'label']).size())
print(test_data.get_label5().groupby(['protocol_type', 'label']).size())

protocol_type  label 
icmp           attack     6982
               normal     1309
tcp            attack    49089
               normal    53600
udp            attack     2559
               normal    12434
dtype: int64
protocol_type  label 
icmp           attack      950
               normal       93
tcp            attack    11038
               normal     7842
udp            attack      845
               normal     1776
dtype: int64
protocol_type  label 
icmp           dos        2847
               normal     1309
               probe      4135
tcp            dos       42188
               normal    53600
               probe      5857
               r2l         995
               u2r          49
udp            dos         892
               normal    12434
               probe      1664
               u2r           3
dtype: int64
protocol_type  label 
icmp           dos        706
               normal      93
               probe      240
               r2l          4
tcp      

In [36]:
# stampa la descrizione delle colonne binarie e trasponi i risultati
print(train_data.get_label2()[binary_features].describe().T)
print(test_data.get_label2()[binary_features].describe().T)


                   count      mean       std  min  25%  50%  75%  max
land            125973.0  0.000198  0.014086  0.0  0.0  0.0  0.0  1.0
logged_in       125973.0  0.395736  0.489010  0.0  0.0  0.0  1.0  1.0
root_shell      125973.0  0.001342  0.036603  0.0  0.0  0.0  0.0  1.0
su_attempted    125973.0  0.001103  0.045154  0.0  0.0  0.0  0.0  2.0
is_host_login   125973.0  0.000008  0.002817  0.0  0.0  0.0  0.0  1.0
is_guest_login  125973.0  0.009423  0.096612  0.0  0.0  0.0  0.0  1.0
                  count      mean       std  min  25%  50%  75%  max
land            22544.0  0.000311  0.017619  0.0  0.0  0.0  0.0  1.0
logged_in       22544.0  0.442202  0.496659  0.0  0.0  0.0  1.0  1.0
root_shell      22544.0  0.002440  0.049334  0.0  0.0  0.0  0.0  1.0
su_attempted    22544.0  0.000266  0.021060  0.0  0.0  0.0  0.0  2.0
is_host_login   22544.0  0.000488  0.022084  0.0  0.0  0.0  0.0  1.0
is_guest_login  22544.0  0.028433  0.166211  0.0  0.0  0.0  0.0  1.0
